<a href="https://colab.research.google.com/github/fumio125/meshroom_colab/blob/main/meshroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meshroom on Google Colab

参考：「他人のGPUで"大規模"フォトグラメトリしたい！！」
https://qiita.com/kotauchisunsun/items/5777c94faabdef1523c9


**注意：実行前に、GPUランタイムに接続していることを確認してください。**

無料版Colabは、GPUランタイムの接続可能時間に制限があります。これを超えたら、`n`時間くらい待ってから再挑戦してください。`n`はサーバの込み具合により変わるらしいですが、一説によると12～24時間くらいらしいです。

無料枠を超えないために、入力画像数を数十枚程度に抑えるのが良いでしょう（方法1-1を使う場合は除く）。

## 1. Google Drive を使う版
1. Google Driveの「マイドライブ」直下に「meshroom」というフォルダを作る
2. フォルダ「meshroom」の下に「images」というフォルダを作り、その中に入力画像を入れる
3. 以下いずれかのセルを実行してしばし待つ（時間がかかりそうな場合はたまにアクセスする）
4. meshroom/results以下に結果が出力される

### 1-1. Google Drive に数GB以上の余裕がある場合
Google Colabの制約で途中でランタイムが落ちても、次のセルを起動すれば途中からやりなおせます

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pwd
!wget https://github.com/alicevision/Meshroom/releases/download/v2021.1.0/Meshroom-2021.1.0-linux-cuda10.tar.gz
!mkdir "./meshroom_exe"
!tar xf "./Meshroom-2021.1.0-linux-cuda10.tar.gz" -C "./meshroom_exe" --strip-components 1

!mkdir "drive/MyDrive/meshroom/result"
!mkdir "drive/MyDrive/meshroom/log"

!./meshroom_exe/meshroom_batch \
    --input "/content/drive/MyDrive/meshroom/images" \
    --output "/content/drive/MyDrive/meshroom/result" \
    --cache "/content/drive/MyDrive/meshroom/cache" \
    --forceStatus \
    | tee "/content/drive/MyDrive/meshroom/log/"`date '+%Y%m%d%H%M%S'`".log"

!rm -rf "drive/MyDrive/meshroom/cache"

### 1-2. Google Drive の容量に余裕がない場合
Google Colabの制約で途中でランタイムが落ちたら、最初からやり直しになります

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pwd
!wget https://github.com/alicevision/Meshroom/releases/download/v2021.1.0/Meshroom-2021.1.0-linux-cuda10.tar.gz
!mkdir "./meshroom_exe"
!tar xf "./Meshroom-2021.1.0-linux-cuda10.tar.gz" -C "./meshroom_exe" --strip-components 1

!mkdir "drive/MyDrive/meshroom/result"
!mkdir "drive/MyDrive/meshroom/log"

!./meshroom_exe/meshroom_batch \
    --input "/content/drive/MyDrive/meshroom/images" \
    --output "/content/drive/MyDrive/meshroom/result" \
    --forceStatus \
    | tee "/content/drive/MyDrive/meshroom/log/"`date '+%Y%m%d%H%M%S'`".log"


## 2. Google drive を使わない版
1. 入力画像を、手元のPCの「images」フォルダ内に入れる
2. 左にあるフォルダマークを押し、「images」フォルダを「ファイル」内にドラッグアンドドロップ
3. 次のセルを実行してしばし待つ（時間がかかりそうな場合はたまにアクセスする
4. 「results」以下に結果が出力されるので、右クリック→ダウンロード


In [ ]:
# Google Colabの制約で途中でランタイムが落ちたら、最初からやり直しになります
# ランタイムとともに結果も消えるので、計算が終わったらすぐに結果をダウンロードしてください

!pwd
!wget https://github.com/alicevision/Meshroom/releases/download/v2021.1.0/Meshroom-2021.1.0-linux-cuda10.tar.gz
!mkdir "./meshroom_exe"
!tar xf "./Meshroom-2021.1.0-linux-cuda10.tar.gz" -C "./meshroom_exe" --strip-components 1

!mkdir "./result"
!mkdir "./log"

!./meshroom_exe/meshroom_batch \
    --input "./images" \
    --output "./result" \
    --forceStatus \
    | tee "./log/"`date '+%Y%m%d%H%M%S'`".log"


## 復元が終わったら
出来上がった.objファイルをMeshLabで読み込んで眺めましょう。

https://www.meshlab.net/

## Tips

* 「たまにアクセスする」ためのバッチファイル

  * https://qiita.com/kotauchisunsun/items/5777c94faabdef1523c9 のものを改変。
  * Windows用、64 bit Google Chrome用です。拡張子"bat"で保存してください。
  * 起動後、colabファイルのURL（```https://colab.research.google.com/drive/...```）を入力してください。


```
echo off
set /P URL="Copy-and-paste Google Colab URL: "
:LOOP
    "C:\Program Files\Google\Chrome\Application\chrome.exe" %URL%
    timeout 3600
goto :LOOP

```



## 既知の問題

* 最新版（Meshroom 2023.2.0）はバッチ出力のテクスチャがデフォルトでexr形式になる。
  * exrのテクスチャはMeshLabで読み込めないので、オプションを変えてpngか何かで出力したいものの、コマンドライン上でテクスチャ形式の指定方法がわからないので過去のバージョンを使っています。
  * Blenderで眺めるならexrテクスチャでもOK
